## <font color='green'> <div align="center">In the name of God </div></font>

### <font color='red'> Author: Sayed Kamaledin Ghiasi-Shrirazi <a href="http://profsite.um.ac.ir/~k.ghiasi">(http://profsite.um.ac.ir/~k.ghiasi)</a> </font>

# GMM Exact Bayes Classifier on MNIST with reg = 0.1

##### Importing general modules.

In [1]:
import numpy as np
from sklearn import mixture
import scipy.io as sio
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.special import logsumexp
from tqdm import tqdm

### Preparing matplotlib to genrate high-quality images for the paper

In [2]:
%matplotlib inline
mpl.rcParams['figure.dpi']= 600

In [3]:
C  = 10
K = 6
L = C * K
width = 28
height = 28
reg = 0.1

### Load Training and Testing Data and Normalize them

In [4]:
XTrain = sio.loadmat ('../../datasets/mnist/MnistTrainX')['MnistTrainX']
XTrain = XTrain / 255
yTrain = sio.loadmat ('../../datasets/mnist/MnistTrainY')['MnistTrainY']
XTest = sio.loadmat ('../../datasets/mnist/MnistTestX')['MnistTestX']
XTest = XTest / 255
yTest = sio.loadmat ('../../datasets/mnist/MnistTestY')['MnistTestY']
N, dim = XTrain.shape
N = 60000
XTrain = XTrain[0:N,:]
yTrain = yTrain[0:N]

### Loading GMM data computed on Google Colab

In [5]:
clusters = np.zeros(N, dtype = int)
with open('GMM_MNIST_6.txt', 'r') as f:
    for i in range (N):
        s = f.readline()
        clusters[i] = int(s)

### Compute means and covariances

In [6]:
means = np.zeros([L, dim])
cov = np.zeros([L, dim, dim])
for i in range (L):
    idx = clusters == i
    means[i,:] = np.mean(XTrain[idx,:], axis = 0)
    cov[i,:,:] = np.cov (XTrain[idx,:].T)    

### Compute priors

In [7]:
Nck = np.bincount(clusters)
Pck = Nck / N

In [15]:
logProb = np.zeros ([N,L])
for i in range (L):
    print (i, end='.')
    eigvals, eigvecs = np.linalg.eig(cov[i,:,:])
    eigvals = np.real(eigvals) + reg
    eigvecs = np.real(eigvecs)
    sigmaHalfInv = eigvecs @ np.diag(np.sqrt(1/eigvals)) @ eigvecs.T
    X = (XTrain - means[i,:]) @ sigmaHalfInv
    X = X ** 2
    d2 = np.sum(X, axis = 1)
    logProb[:,i] = np.log(Pck[i]) - 1 / 2 * d2 -1/2 * np.sum(np.log(eigvals))
print ('')    
logProbClass = np.zeros([N,C])
for n in range (N):
    for c in range (C):
        logProbClass[n,c] = logsumexp(logProb[n,c*K:(c+1)*K])
prediction = np.argmax (logProbClass, axis = 1)
score = np.sum(prediction == yTrain.squeeze())/ N
print ('Training Accuracy = ', score)

0.1.2.3.4.5.6.7.8.9.10.11.12.13.14.15.16.17.18.19.20.21.22.23.24.25.26.27.28.29.30.31.32.33.34.35.36.37.38.39.40.41.42.43.44.45.46.47.48.49.50.51.52.53.54.55.56.57.58.59.
Training Accuracy =  0.9793333333333333


In [16]:
NTest = XTest.shape[0]
logProb = np.zeros ([NTest,L])
for i in tqdm(range (L)):
    eigvals, eigvecs = np.linalg.eig(cov[i,:,:])
    eigvals = np.real(eigvals) + reg
    eigvecs = np.real(eigvecs)
    sigmaHalfInv = eigvecs @ np.diag(np.sqrt(1/eigvals)) @ eigvecs.T
    X = (XTest - means[i,:]) @ sigmaHalfInv
    X = X ** 2
    d2 = np.sum(X, axis = 1)
    logProb[:,i] = np.log(Pck[i]) - 1 / 2 * d2 -1/2 * np.sum(np.log(eigvals))
prediction = np.argmax (logProb, axis = 1) // K
score = np.sum(prediction == yTest.squeeze())/ NTest
print ('Testing Accuracy = ', score)

100%|██████████████████████████████████████████████████████████████████████████████████| 60/60 [02:12<00:00,  2.70s/it]


Testing Accuracy =  0.9689


### <font color='red'> Author: Sayed Kamaledin Ghiasi-Shrirazi <a href="http://profsite.um.ac.ir/~k.ghiasi">(http://profsite.um.ac.ir/~k.ghiasi)</a> </font>